# Footprint of Travel
- This looks like a helpful platform: https://geops.com/de/solution
- Specifically for real-time: https://developer.geops.io/apis/realtime

# Footprint of train travel
Spatially and temporally granular assessment of footprint
- Estimate direct power use of the train
  - Get the train model
  - Technical specifications of trains
  - Estimate how full the train will be
  - EMaps or CO2Maps signal to determine the carbon footprint

In [164]:
import requests

### Play with the CO2Map API
See https://api.co2map.de/docs#/ Ok, this is very straight-forward to use.

In [177]:
def query_co2map_api(
    federal_state: str,
    start = "2024-03-01",
    end = "2024-03-02",
    country: str = "DE",
    kind: str = "ConsumptionIntensityPreliminary",
):
    r = requests.get(f'https://api.co2map.de/{kind}/?state={federal_state}&country={country}&start={start}&end={end}')
    return r.json()

In [178]:
signal = query_co2map_api("BW")
signal['Consumption-based Intensity (preliminary)']

### Geops API
This looks a bit tricker to navigate but super cool 🤯
- For real-time train information https://developer.geops.io/apis/realtime
- Here we can get the routes (on varying levels of detail): https://developer.geops.io/apis/routing

In [180]:
r = requests.get(
    url = 'https://api.geops.io/routing/v1/?via=Hamburg%7CK%C3%B8benhavn%20H&mot=rail&graph=gen2',
    headers = {
        'accept': 'application/json',
        'Authorization': '5cc87b12d7c5370001c1d655f8be4df3e3c745f8b14117118ab6cdf4'
    }
)

In [181]:
r.json()

{'features': [{'geometry': {'coordinates': [[10.0064358205, 53.5531997932],
     [10.0028838204, 53.5560326932],
     [9.9950354573, 53.5585083528],
     [9.9885215202, 53.5615458931],
     [9.9835648006, 53.5639118882],
     [9.96779892, 53.563830093],
     [9.9487330249, 53.5619103896],
     [9.9416578052, 53.5642537757],
     [9.9375445442, 53.5662676646],
     [9.9342073196, 53.5688839929],
     [9.9185498191, 53.5896461927],
     [9.9073847189, 53.5956756926],
     [9.9013495939, 53.5986505901],
     [9.8933930187, 53.6026864925],
     [9.8679643183, 53.6157075924],
     [9.8391023178, 53.6304089922],
     [9.8138832174, 53.644350492],
     [9.7973425171, 53.6552587919],
     [9.7602002164, 53.6752948917],
     [9.7152910157, 53.6970014914],
     [9.6592630143, 53.7542255909],
     [9.6451670146, 53.8045036611],
     [9.6460951829, 53.8203372828],
     [9.6807427131, 53.8525614907],
     [9.7479260125, 53.9320957907],
     [9.775189985, 53.9641009584],
     [9.824394094, 53.990385

# Footprint of bus travel
- What bus models do bus providers use?
  - What are the technical details of the bus models?
- How full are the buses?
- What routes do they take?
  - Or at least how long are the road connections between the locations?

# Footprint of air travel
- Which model? ✅
  - Where to query technical information on airplane models?
- How much is the footprint affected by weather? 🚧
  - To be determined
- What routes?
  - See if this is captured in the historical data
- How full are the planes? 
  - See if this is captured in the historical data

## The Flight Radar 24 API - wooooah

### Playing around with airport details

In [3]:
from FlightRadar24 import FlightRadar24API
from datetime import datetime
fr_api = FlightRadar24API()

In [12]:
airports = fr_api.get_airports()

In [33]:
# Vienna International Airport
VIE = [airport for airport in airports if airport.__getattribute__("iata") == 'VIE']
VIE

[<(LOWW) Vienna International Airport - Altitude: 600 - Latitude: 48.110271 - Longitude: 16.569719>]

In [39]:
fr_api.get_airport_details("vie")["airport"]["pluginData"].keys()

dict_keys(['details', 'flightdiary', 'schedule', 'weather', 'aircraftCount', 'runways'])

In [41]:
fr_api.get_airport_details("vie")["airport"]["pluginData"]["details"].keys()

dict_keys(['name', 'code', 'delayIndex', 'stats', 'position', 'timezone', 'url', 'airportImages', 'visible'])

In [59]:
fr_api.get_airport_details("vie")["airport"]["pluginData"]["schedule"]["arrivals"]["data"][0]["flight"].keys()

dict_keys(['identification', 'status', 'aircraft', 'owner', 'airline', 'airport', 'time'])

### Query flights - WIP
- Step 1: Given some details of the flight, we want to be able to get the ID
  - We can get the ID, but this is not a Flight object, only a string - and not sufficient to get what we want??
\n
- We could also try to query from the airport information ...

In [81]:
example_flight = fr_api.search(
    "HLR555"
)
example_flight

In [69]:
identification = example_flight["live"][0]["id"]

### Get flight details from arrivals
- potentially not the best way to get historical data, because I think this looks at realtime data
- -> also there seems to be a way to get historical data (get_history_data), to be explored 🧐

In [160]:

def get_flight_info(
    arrival_airport: str, # IATA code, e.g. VIE for Vienna or LHR for London Heathrow
    callsign: str,
):
    arrival_data = fr_api.get_airport_details(arrival_airport)["airport"]["pluginData"]["schedule"]["arrivals"]["data"]
    flights = [flight for flight in arrival_data if flight["flight"]["identification"]["callsign"] == callsign]

    if len(flights) == 0:
        return None
    elif len(flights) > 1:
        raise ValueError("Multiple flights found")
    
    return flights[0]

In [118]:
arrival_data = fr_api.get_airport_details("vie")["airport"]["pluginData"]["schedule"]["arrivals"]["data"]

In [139]:
flight = get_flight_info("vie", "RYR6QV")
flight

We can obtain some interesting information from this flight:
- Estimated and later real duration
- Operator (could give us an indication of typical loads)
- Aircraft details

What I haven't figured out yet
- How full the airplane is, not sure if FlightRadar even has this information
- How long the trip is/was. We would just do distance between airports as an approximation (we can query this information from the API but the actual travelled distance could be useful and I think that flightradar tracks this 🧐)

In [154]:
flight['flight']['identification']

{'id': None,
 'row': 5479612370,
 'number': {'default': 'FR9888', 'alternative': 'LW9888'},
 'callsign': 'RYR6QV',
 'codeshare': None}

In [155]:
flight['flight']['aircraft']

{'model': {'code': 'A320', 'text': 'Airbus A320-214'},
 'registration': '9H-LOQ',
 'country': {'id': 136, 'name': 'Malta', 'alpha2': 'MT', 'alpha3': 'MLT'},
 'hex': '4D22C8',
 'restricted': False,
 'serialNo': None,
 'age': {'availability': True},
 'availability': {'serialNo': True, 'age': True}}

In [156]:
flight['flight']['time']

{'scheduled': {'departure': 1710083100, 'arrival': 1710089400},
 'real': {'departure': None, 'arrival': None},
 'estimated': {'departure': 1710083700, 'arrival': None},
 'other': {'eta': None, 'duration': None}}

{'live': False,
 'text': 'Scheduled',
 'icon': None,
 'estimated': None,
 'ambiguous': False,
 'generic': {'status': {'text': 'scheduled',
   'type': 'arrival',
   'color': 'gray',
   'diverted': None},
  'eventTime': {'utc': None, 'local': None}}}